-----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Librerias

In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
from utils.utilities import Tiempo
import statistics
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que extrae los timestamps de inicio y fin de cada intentos de nivel
#### También extrae las estrellas obtenidas en cada intentos y cuantas veces se han intentado todos los niveles por cada jugador 

In [ ]:
def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
        
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1
                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que resta los timestamps de inicio y fin de cada intento de nivel para sacar la diferencia de tiempo

In [ ]:
def tiempoPorNiveles_Jugador(data):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    timeDifference = Tiempo(times["ini"], times["fin"])
                    if level in tiemposJugados[player]:
                        tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                    else:
                        tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
    return tiemposJugados

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que devuelve el tiempo medio necesario para completar un nivel y un diccionario con el tiempo empleado por cada jugador hasta completar el nivel
(El tiempo necesario para cada jugador es la suma de todos los tiempos hasta conseguir completar el nivel)
* También devuelve la media de estrellas y una lista de los niveles ordenados

In [ ]:
#TAMBIEN DEVUELVE EL TIEMPO INDIVIDUAL DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getMediaTiempoPorNivel(tiempos, soloPrimerExito = True, tiemposOrdenados = False):
    medias = defaultdict(list)
    mediasEstrellas = defaultdict(list)
    tiempoCompletarNivelIndividual = defaultdict(defaultdict)
    
    for player in tiempos:
        for level in tiempos[player]:
            tAux = Tiempo("0s")
            for t in tiempos[player][level]:
                if t["stars"] != -1:
                    if level in medias:
                        medias[level].append(int(tAux + t["time"]))
                        mediasEstrellas[level].append(int(t["stars"]))
                    else:
                        medias[level] = [int(tAux + t["time"])]
                        mediasEstrellas[level] = [int(t["stars"])]
                        
                    tiempoCompletarNivelIndividual[player][level] = tAux + t["time"]
                    if soloPrimerExito:
                        break
                elif t["stars"] == 0:
                    print("ALERTA")
                else:
                    tAux += t["time"]
    for m in medias:
        medias[m] = Tiempo(str(int(round(statistics.mean(medias[m]), 0))) + "s")
        
    for m in mediasEstrellas:
        mediasEstrellas[m] = statistics.mean(mediasEstrellas[m])
    
    listaNiveles = medias.keys()
    
    if tiemposOrdenados:
        medias = sorted(medias.items(), key=lambda x: x[1])
        mediasEstrellas = sorted(mediasEstrellas.items(), key=lambda x: x[1])
    else:
        medias = list(medias.items())
        mediasEstrellas = list(mediasEstrellas.items())
    
    return {"mediaTiempos" : medias, "mediaEstrellas" : mediasEstrellas, "listaNiveles" : listaNiveles, "tiemposIndividuales" : tiempoCompletarNivelIndividual}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve un diccionario con clave: id Jugador y valor: ultimo nivel alcanzado

In [ ]:
def getUltimoNivelAlcanzado(tiempos):
    ultNivel = defaultdict()
    for player in tiempos:
        for level in tiempos[player]:
            ultNivel[player] = level
    return ultNivel

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve los intentos necesarios de cada jugador para completar un nivel al igual que los intentos medios

In [ ]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Transforma un diccionario de diccionarios en un diccionario de arrays
* Se utiliza para tener una lista de todos los tiempos de los usuarios cuando tenemos un diccionario dentro de otro con primera clave nombre usuario y segunda clave codigo del nivel

In [ ]:
def extraerArray(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append(nested_v)

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(list)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key].append(value)
    
    return nested_dict

In [ ]:
def extraerArrayConNombres(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append({"name" : k, "valor" : nested_v})

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(defaultdict)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key][value["name"]] = value["valor"]
    
    return nested_dict

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Recibe un diccionario con clave nivel y valor una lista de tiempos y parsea los tiempos a integer

In [ ]:
def parseTiemposDictToInteger(data_dict):
    parsed_dict = {}

    for key, value in data_dict.items():
        parsed_list = []
        for element in value:
            parsed_list.append(int(element))
        parsed_dict[key] = parsed_list

    return parsed_dict

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que genera los boxplots recibiendo un diccionario con clave nivel y valor una lista de enteros

In [ ]:
def create_boxplots(data_dict):
    # Group levels by category
    category_dict = {}
    for level, data in data_dict.items():
        category = level.split('_')[0]
        if category in category_dict:
            category_dict[category].append(data)
        else:
            category_dict[category] = [data]
    
    # Create boxplots for each category and add relevant information
    boxplots = []
    labels = []
    for i, (category, data_list) in enumerate(category_dict.items()):
        fig, ax = plt.subplots()
        bp = ax.boxplot(data_list, showfliers=True, patch_artist=True, flierprops={'marker': '+', 'markerfacecolor': '#738FA7'})
        ax.set_title(category)

        color = '#738FA7'
        for patch in bp['boxes']:
            patch.set_facecolor(color)
        
        boxplots.append(bp)
        labels.append(category)
    
    # Set tick labels
    fig, ax = plt.subplots()
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels)
    ax.tick_params(axis='x', labelrotation=45)
    
    return boxplots


In [ ]:
def getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles):
    cuantosHanLlegadoAlNivel = defaultdict()
    
    ultNivelCopia = ultNivelCompletado.copy()

    #Contamos cuanta gente ha llegado hasta cada nivel
    for nivel in niveles:
        cuantosHanLlegadoAlNivel[nivel] = len(ultNivelCopia)
        
        keys_to_delete = [k for k, v in ultNivelCopia.items() if v == nivel]
        for k in keys_to_delete:
            del ultNivelCopia[k]

    return cuantosHanLlegadoAlNivel

In [ ]:
def generateChartNivelesAlcanzados(niveles, ultNivelCompletado):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)

    category_dict = {}
    for level, value in cuantosHanLlegadoAlNivel.items():
        category = level.split('_')[0]
        level_number = int(level.split('_')[-1])
        if category in category_dict:
            category_dict[category].append((level_number, value))
        else:
            category_dict[category] = [(level_number, value)]
    for category, levels in category_dict.items():
        category_dict[category] = sorted(levels, key=lambda x: x[0])

    # Select the last level for each category
    last_levels = {}
    for category, levels in category_dict.items():
        last_level = levels[-1][0]
        last_levels[category] = levels[-1][1]

    # Create bar chart and add labels
    fig, ax = plt.subplots()
    ax.bar(last_levels.keys(), last_levels.values(), color="#738FA7")
    ax.set_xticklabels(last_levels.keys(), rotation=45, ha='right')
        
    plt.show()

In [ ]:
def generateChartNivelesAlcanzados2(niveles, ultNivelCompletado):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]
    
    df = pd.DataFrame({"levels" : list(ultNivelCat.keys()), "nJugadores" : list(ultNivelCat.values())})
    fig = px.bar(df, x="levels", y='nJugadores', labels={'levels':'Categorías', 'nJugadores':'Numero Jugadores'})
    fig.update_layout(plot_bgcolor='#C3CEDA')
    fig.update_traces(marker_color='#738FA7', hovertemplate='<b>Numero Jugadores: %{y}</b>')
    #fig.write_json("./plots/categorias.json")
    return fig


In [ ]:
def getChartPorcentajeCategorias(niveles, ultNivelCompletado, total_players):

    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)

    ultimoNivelCat = {}
    for level in niveles:
        category = level.split("_")[0]
        ultimoNivelCat[category] = level
    
    percentages = {}
    for ultNivel in ultimoNivelCat.values():
        percentages[ultNivel] = (cuantosHanLlegadoAlNivel[ultNivel]/total_players)*100
    
    fig, ax = plt.subplots()
    ax.bar(range(len(percentages)), list(percentages.values()), align='center', color="#738FA7")
    ax.set_xticks(range(len(percentages)))
    ax.set_xticklabels(map(lambda c: c.split("_")[0], list(percentages.keys())), rotation=45, ha='right')
    ax.set_ylim((0, 100))
    ax.set_ylabel('Porcentaje Jugadores')
    ax.set_xlabel('Categorías')
    ax.set_title('Categorías Superadas')
    
    return ax

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
#JSONFile = open('../Web/datos/1/trazasOrdenadas.json')
JSONFile = open('./data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

In [ ]:
tiemposIntentosJugadores = tiempoPorNiveles_Jugador(resultados_Tiempos_Nivel_Jugador["tiempos"])
pd.DataFrame(tiemposIntentosJugadores)
#pd.DataFrame(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"])

In [ ]:
soloPrimerExito = True
tiemposOrdenados = False
tiemposMedios = getMediaTiempoPorNivel(tiemposIntentosJugadores, soloPrimerExito, tiemposOrdenados)
#pd.DataFrame(tiemposMedios["mediaTiempos"])
#pd.DataFrame(tiemposMedios["mediaEstrellas"])
#pd.DataFrame(tiemposMedios["listaNiveles"])
#pd.DataFrame(tiemposMedios["tiemposIndividuales"])

In [ ]:
ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposMedios["tiemposIndividuales"])
#print(ultNivelAlcanzado)

In [ ]:
intentosOrdenados = False
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)
#pd.DataFrame(intentosMedios_Individual["intentosMedios"])
#pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

In [ ]:
tiemposList = extraerArray(tiemposMedios["tiemposIndividuales"])
intentosList = extraerArray(intentosMedios_Individual["intentosIndividual"])
#pd.DataFrame.from_dict(tiemposList, orient='index').transpose()
#pd.DataFrame.from_dict(intentosList, orient='index').transpose()

In [ ]:
intentosListNombres = extraerArrayConNombres(intentosMedios_Individual["intentosIndividual"])
tiemposListNombres = extraerArrayConNombres(tiemposMedios["tiemposIndividuales"])
#pd.DataFrame.from_dict(intentosListNombres, orient='index').transpose()
#pd.DataFrame.from_dict(tiemposListNombres, orient='index').transpose()

In [ ]:
def parseTiemposDictConNombresToInteger(tiemposDict):
    for l in tiemposDict:
        for j in tiemposDict[l]:
            tiemposDict[l][j] = int(tiemposDict[l][j])
    return tiemposDict

In [ ]:
fig = generateChartNivelesAlcanzados2(tiemposMedios["listaNiveles"], ultNivelAlcanzado)
fig.show()

In [ ]:
#getChartPorcentajeCategorias(tiemposMedios["listaNiveles"], ultNivelAlcanzado, len(ultNivelAlcanzado))

In [ ]:
#boxplotsIntentos = create_boxplots(intentosList)

In [ ]:
#boxplotsTiempos = create_boxplots(parseTiemposDictToInteger(tiemposList))

In [ ]:
def create_boxplots3(data_dict, titulo):
    categorias = defaultdict(defaultdict)
    for d in data_dict:
        categorias[d.split("_")[0]][d] = data_dict[d]

    boxplots = []
    for c in categorias:
        df = pd.DataFrame.from_dict(categorias[c], orient="index")
        df = df.reset_index()
        df = df.rename(columns={'index' : 'Niveles'})
        df_melted = df.melt(id_vars=['Niveles'], var_name='Jugador', value_name=titulo)
        # create boxplot
        fig = px.box(df_melted, x='Niveles', y=titulo, hover_name='Jugador')
        fig.update_layout(plot_bgcolor='#C3CEDA')
        fig.update_traces(marker_color='#738FA7', hovertemplate='<b>%{hovertext}</b><br>' + titulo + ': %{y}')
        #fig.write_json("./plots/"+c + "_"+ titulo +".json")
        boxplots.append(fig)
    return boxplots

In [ ]:
boxsIntentos= create_boxplots3(intentosListNombres, 'Intentos')
for b in boxsIntentos:
    #b.show()
    None

In [ ]:
boxsTiempos = create_boxplots3(parseTiemposDictConNombresToInteger(tiemposListNombres), "Tiempo(s)")
for b in boxsTiempos:
    #b.show()
    None

In [ ]:
def getChartComparativas(niveles, tiemposMedios, ultNivelCompletado, jugClase):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]

    for c in ultNivelCat:
        ultNivelCat[c] = round((ultNivelCat[c]/jugClase)*100, 2)

    categorias = defaultdict(defaultdict)
    for d in tiemposMedios:
        categorias[(" ".join(d[0].split("_")[:-1])).capitalize()][d[0]] = int(d[1])

    ### Actualizar datos globales
    JSONFile = open('datosGlobales.json')
    datosGlobales = json.load(JSONFile)
    JSONFile.close()

    if( 1 not in datosGlobales["institutos"]):
        jugTotales = jugClase + datosGlobales["numeroAlumnos"]
        for c in datosGlobales["porcentaje"]:
            if c in ultNivelCat:
                datosGlobales["porcentaje"][c] = (datosGlobales["porcentaje"][c]*datosGlobales["numeroAlumnos"]/jugTotales) + (ultNivelCat[c]*jugClase/jugTotales)

        
        datosGlobales["numeroAlumnos"] = jugTotales        
        datosGlobales["institutos"].append(1)

        for c in datosGlobales["tiempoMedio"]:
            if c in categorias:
                for l in datosGlobales["tiempoMedio"][c]:
                    if l in categorias[c]:
                        datosGlobales["tiempoMedio"][c][l] = (datosGlobales["tiempoMedio"][c][l]*datosGlobales["numeroAlumnos"]/jugTotales) + (categorias[c][l]*jugClase/jugTotales)
                        
        with open('datosGlobales.json', 'w') as json_file:
            json.dump(datosGlobales, json_file)


    print(ultNivelCat)
    fig = go.Figure(data=[
        go.Bar(name="Clase", x=list(ultNivelCat.keys()), y=list(ultNivelCat.values()), marker_color="#738FA7",
               hovertemplate='<b>%{y}%</b>'),
        go.Bar(name="Global", x=list(datosGlobales["porcentaje"].keys()), y=list(datosGlobales["porcentaje"].values()), marker_color="#0C4160",
               hovertemplate='<b>%{y}%</b>'),
    ])
    fig.update_layout(plot_bgcolor='#C3CEDA')
    fig.update_layout(barmode='group')
    fig.update_layout(title_text='Porcentaje Categorías Superadas VS Global')
    #fig.write_json("./plots/porcentajeCategorias.json")
    fig.show()


    print(categorias)
    for c in datosGlobales["tiempoMedio"]:
        if c in categorias:
            fig = go.Figure(data=[
                go.Bar(name = "Clase", x=list(categorias[c].keys()), y=list(categorias[c].values()), marker_color="#738FA7",
                    hovertemplate='<b>%{y}s</b>'),
                go.Bar(name = "Global", x=list(datosGlobales["tiempoMedio"][c].keys()), y=list(datosGlobales["tiempoMedio"][c].values()), marker_color="#0C4160",
                    hovertemplate='<b>%{y}s</b>')
            ])
            fig.update_layout(plot_bgcolor='#C3CEDA')
            fig.update_layout(barmode='group')
            fig.write_json("./plots/" + c.split(" ")[0].lower() + "_mediaTiemposComparativa.json")
            fig.show()

In [ ]:
getChartComparativas(tiemposMedios["listaNiveles"], tiemposMedios["mediaTiempos"], ultNivelAlcanzado, len(ultNivelAlcanzado))


In [ ]:
print(tiemposMedios["mediaTiempos"])

In [ ]:
JSONFile = open('./data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

for e in rawData:
    name = e["actor"]["name"]
    if name == "smcq":
        print(e)